<a href="https://colab.research.google.com/github/farzanehnia/PYOMO/blob/main/Power_Generation_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -q pyomo

     |████████████████████████████████| 9.6 MB 15.6 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 


In [3]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [10]:
#inputs
!rm inputs.xlsx

!wget --show-progress --continue -O /content/inputs.xlsx https://raw.github.com/farzanehnia/PYOMO/main/inputs.xlsx

dataGen = pd.read_excel('inputs.xlsx', sheet_name='gen')
dataLoad = pd.read_excel('inputs.xlsx', sheet_name='load')

Ng = len(dataGen)

--2022-07-03 09:52:00--  https://raw.github.com/farzanehnia/PYOMO/main/inputs.xlsx
Resolving raw.github.com (raw.github.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/farzanehnia/PYOMO/main/inputs.xlsx [following]
--2022-07-03 09:52:01--  https://raw.githubusercontent.com/farzanehnia/PYOMO/main/inputs.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9455 (9.2K) [application/octet-stream]
Saving to: ‘/content/inputs.xlsx’

/content/inputs.xls 100%[===================>]   9.23K  --.-KB/s    in 0s      

2022-07-03 09:52:01 (63.0 MB/s) - ‘/content/inputs.xls

In [16]:
import re
dataLoad['Load Demand'] = dataLoad['Load Demand'].astype('str').str.replace('\D', "").astype('float')
dataGen['Power Generation'] = dataGen['Power Generation'].astype('str').str.replace('\D', "").astype('float')


dataGen.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,id,Cost,Power Generation
0,0,0.10,20.0
1,1,0.05,10.0
2,2,0.30,40.0
3,3,0.40,50.0
4,4,0.01,5.0


In [19]:
#model

model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), bounds=(0, None))
Pg = model.Pg

#constraints
Pg_sum = sum([Pg[g] for g in dataGen.id])
demand_sum = dataLoad['Load Demand'].sum()


model.balance = pyo.Constraint(expr = Pg_sum == demand_sum)


<bound method Component.pprint of <pyomo.core.base.PyomoModel.ConcreteModel object at 0x7f7bbc25a410>>